In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

dropbox_folder = '/Users/hanyin/Dropbox (CSU Fullerton)/Research/Flies/'
project_folder = '/Kshama&Arnie -- statistical analysis/summary - Fall 2022/'
os.chdir(dropbox_folder + project_folder)

### Part 1: Load Data File

In [2]:
from numbers_parser import Document

doc = Document('data/data_raw.numbers')
sheets = doc.sheets

df_names = ['mating','diet1','diet2','diet3']
for num in range(len(df_names)):    
    sheet_name = sheets[num].name
    data = sheets[num].tables[0].rows(values_only = True)
    globals()[df_names[num]] = pd.DataFrame(data[1:], columns = data[0])
    
    shape = globals()[df_names[num]].shape
    print(f'Sheet {num}: {sheet_name}    -->    {df_names[num]} ({shape[0]}, {shape[1]})')

Sheet 0: SSMI Mating    -->    mating (1736, 12)
Sheet 1: Diet 1    -->    diet1 (1680, 9)
Sheet 2: Diet 2: Diet Switch    -->    diet2 (1728, 9)
Sheet 3: Diet 3: Yeast Variation    -->    diet3 (896, 9)


### Part 2: Format and Clean
#### (a) SSMI Mating

In [3]:
def clean_nan(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

mapping = {'NA':'NaN', 'N/A':'NaN', 'None':'NaN'}

In [4]:
## clean col Eggs
mating['Eggs'] = [clean_nan(str(egg), mapping) for egg in mating['Eggs']]

## clean col Larval
mating['Larval Viability'] = [clean_nan(str(larva), mapping) for larva in mating['Larval Viability']]

## rename fungal treatment
mating['Treatment'] = mating['Treatment'].replace('Fungal', 'Inoculated')

#### (b) Diet 1 & 2 & 3

In [5]:
def clean_diet(diet):
    return ['/'.join([''.join(ingredient.strip()[0] for ingredient in meal.split('+')) \
            for meal in meals.split(',')][:2]) \
            for meals in diet]

In [6]:
## rename col Diet
diet1['Diet'] = clean_diet(diet1['Diet'])
diet2['Diet'] = clean_diet(diet2['Diet'])

## cut off col Day
diet1 = diet1[diet1['Day'] <= 12]
diet2 = diet2[diet2['Day'] <= 9]
diet3 = diet3[diet3['Day'] <= 12]

### Part 3: Write Out

In [7]:
writer = pd.ExcelWriter('data/data_clean.xlsx', engine = 'xlsxwriter')

for sheet in sheets:
    globals()[df_names[num]].to_excel(writer, sheet_name = sheet.name.replace(':',''))

writer.save()